In [1]:
#mirror whole ct
import SimpleITK as sitk

img = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii")

flipped_array = sitk.GetArrayFromImage(img)[:, :, ::-1]  
img_flipped = sitk.GetImageFromArray(flipped_array)
img_flipped.CopyInformation(img) 

sitk.WriteImage(img_flipped, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_mirrored.nii")
print("saved")


saved


In [3]:
#mirror mandible
import SimpleITK as sitk

img = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\TotalSegmentator1\mandible.nii.gz")

flipped_array = sitk.GetArrayFromImage(img)[:, :, ::-1]  
img_flipped = sitk.GetImageFromArray(flipped_array)
img_flipped.CopyInformation(img) 

sitk.WriteImage(img_flipped, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_mirrored.nii.gz")
print("saved")


saved


In [8]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# ---------------------- USER PATHS ----------------------
ct1_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii"
ct2_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_mirrored.nii"

mand1_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\TotalSegmentator1\mandible.nii.gz"
mand2_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_mirrored.nii.gz"
# ---------------------------------------------------------


# ---------------------- Load CTs -------------------------
ct1_img = nib.load(ct1_path)
ct2_img = nib.load(ct2_path)

ct1 = ct1_img.get_fdata()
ct2 = ct2_img.get_fdata()

# Normalize both CTs to 0–1 for display consistency
ct1 = (ct1 - np.min(ct1)) / (np.max(ct1) - np.min(ct1))
ct2 = (ct2 - np.min(ct2)) / (np.max(ct2) - np.min(ct2))

spacing = ct1_img.header.get_zooms()


# ---------------------- Load segmentations ----------------
mand1 = nib.load(mand1_path).get_fdata() > 0.5
mand2 = nib.load(mand2_path).get_fdata() > 0.5


# ---------------------- Colors ----------------------------
color_mand1 = (1, 1, 0, 0.45)   # original mandible = yellow
color_mand2 = (0, 1, 1, 0.45)   # mirrored mandible = cyan

# Bone colormap for CT2
overlay_cmap = plt.cm.bone
overlay_cmap.set_under(alpha=0)


# ---------------------- Slice Viewer ----------------------
def show_slice(slice_index, plane, alpha=0.35):

    plt.figure(figsize=(6, 6))

    # ======================= AXIAL =======================
    if plane == 'Axial':
        img1 = np.rot90(ct1[:, :, slice_index])
        img2 = np.rot90(ct2[:, :, slice_index])

        m1 = np.rot90(mand1[:, :, slice_index])
        m2 = np.rot90(mand2[:, :, slice_index])

        aspect = spacing[1] / spacing[0]

    # ======================= CORONAL =======================
    elif plane == 'Coronal':
        img1 = np.flipud(np.rot90(ct1[:, slice_index, :]))
        img2 = np.flipud(np.rot90(ct2[:, slice_index, :]))

        m1 = np.flipud(np.rot90(mand1[:, slice_index, :]))
        m2 = np.flipud(np.rot90(mand2[:, slice_index, :]))

        aspect = spacing[2] / spacing[0]

    # ======================= SAGITTAL =======================
    elif plane == 'Sagittal':
        img1 = np.flipud(np.rot90(ct1[slice_index, :, :]))
        img2 = np.flipud(np.rot90(ct2[slice_index, :, :]))

        m1 = np.flipud(np.rot90(mand1[slice_index, :, :]))
        m2 = np.flipud(np.rot90(mand2[slice_index, :, :]))

        aspect = spacing[2] / spacing[1]


    # ======================= BASE CT1 =======================
    plt.imshow(img1, cmap='gray', origin='lower', aspect=aspect)


    # ======================= CT2 bone overlay (intensity preserved!) =======================
    ct2_rgba = overlay_cmap(img2)
    ct2_rgba[..., 3] = alpha   # set transparency for entire CT2 layer
    plt.imshow(ct2_rgba, origin='lower', aspect=aspect)


    # ======================= Mandible 1 =======================
    overlay1 = np.zeros((*m1.shape, 4))
    overlay1[m1] = (*color_mand1[:3], alpha)
    plt.imshow(overlay1, origin='lower', aspect=aspect)


    # ======================= Mandible 2 =======================
    overlay2 = np.zeros((*m2.shape, 4))
    overlay2[m2] = (*color_mand2[:3], alpha)
    plt.imshow(overlay2, origin='lower', aspect=aspect)


    # ======================= Decoration =======================
    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")

    legend_handles = [
        plt.Line2D([0], [0], color=color_mand1[:3], lw=4, label="Mandible 1"),
        plt.Line2D([0], [0], color=color_mand2[:3], lw=4, label="Mandible 2"),
        plt.Line2D([0], [0], color="gray", lw=4, label="CT1 (base)"),
        plt.Line2D([0], [0], color="black", lw=4, label="CT2 (bone colormap)"),
    ]
    plt.legend(handles=legend_handles, loc='upper right', fontsize=8, frameon=False, labelcolor = "white")

    clear_output(wait=True)
    display(plt.gcf())
    plt.close()



# ---------------------- Widgets ----------------------
plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=ct1.shape[2] // 2,
    min=0,
    max=ct1.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


# update slice limits when plane changes
def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = ct1.shape[2] - 1
        slice_slider.value = ct1.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = ct1.shape[1] - 1
        slice_slider.value = ct1.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = ct1.shape[0] - 1
        slice_slider.value = ct1.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(
    show_slice,
    slice_index=slice_slider,
    plane=plane_dropdown,
    alpha=alpha_slider
)


interactive(children=(IntSlider(value=89, description='Slice:', layout=Layout(width='80%'), max=178), Dropdown…

<function __main__.show_slice(slice_index, plane, alpha=0.35)>

In [1]:
import SimpleITK as sitk

fixed = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii")   
moving = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_mirrored.nii")   

initial_tx = sitk.CenteredTransformInitializer(
    fixed,
    moving,
    sitk.VersorRigid3DTransform(),
    sitk.CenteredTransformInitializerFilter.GEOMETRY
)

reg = sitk.ImageRegistrationMethod()

reg.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
reg.SetMetricSamplingPercentage(0.2, sitk.sitkWallClock)
reg.SetMetricSamplingStrategy(reg.RANDOM)

reg.SetInterpolator(sitk.sitkLinear)

reg.SetOptimizerAsRegularStepGradientDescent(
    learningRate=2.0,
    minStep=1e-4,
    numberOfIterations=200,
    gradientMagnitudeTolerance=1e-6
)

# prevents big rotations that cause the optimizer to flip
reg.SetOptimizerScalesFromPhysicalShift()

reg.SetInitialTransform(initial_tx, inPlace=False)

final_tx = reg.Execute(fixed, moving)

sitk.WriteTransform(final_tx, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\rigid_mirror_to_original.tfm")
print("Saved registration transform")

resampler = sitk.ResampleImageFilter()

resampler.SetReferenceImage(fixed)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetTransform(final_tx)

registered_mirror_CT = resampler.Execute(moving)

sitk.WriteImage(registered_mirror_CT, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_registered_ct.nii.gz")
print("Saved registered CT")

Saved registration transform
Saved registered CT


In [3]:
mand = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_mirrored.nii.gz")

resamp_seg = sitk.ResampleImageFilter()
resamp_seg.SetReferenceImage(fixed)
resamp_seg.SetInterpolator(sitk.sitkNearestNeighbor)
resamp_seg.SetTransform(final_tx)

registered_mand = resamp_seg.Execute(mand)
sitk.WriteImage(registered_mand, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_registered_mirrored.nii.gz")

print("Saved registered mandible")


Saved registered mandible


In [5]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

ct1_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii"
ct2_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_registered_ct.nii.gz"

mand1_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\TotalSegmentator1\mandible.nii.gz"
mand2_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mandible_registered_mirrored.nii.gz"

ct1_img = nib.load(ct1_path)
ct2_img = nib.load(ct2_path)

ct1 = ct1_img.get_fdata()
ct2 = ct2_img.get_fdata()

# Normalize both CTs to 0–1 for display consistency
ct1 = (ct1 - np.min(ct1)) / (np.max(ct1) - np.min(ct1))
ct2 = (ct2 - np.min(ct2)) / (np.max(ct2) - np.min(ct2))

spacing = ct1_img.header.get_zooms()


# ---------------------- Load segmentations ----------------
mand1 = nib.load(mand1_path).get_fdata() > 0.5
mand2 = nib.load(mand2_path).get_fdata() > 0.5


# ---------------------- Colors ----------------------------
color_mand1 = (1, 1, 0, 0.45)   # original mandible = yellow
color_mand2 = (0, 1, 1, 0.45)   # mirrored mandible = cyan

# Bone colormap for CT2
overlay_cmap = plt.cm.bone
overlay_cmap.set_under(alpha=0)


# ---------------------- Slice Viewer ----------------------
def show_slice(slice_index, plane, alpha=0.35):

    plt.figure(figsize=(6, 6))

    # ======================= AXIAL =======================
    if plane == 'Axial':
        img1 = np.rot90(ct1[:, :, slice_index])
        img2 = np.rot90(ct2[:, :, slice_index])

        m1 = np.rot90(mand1[:, :, slice_index])
        m2 = np.rot90(mand2[:, :, slice_index])

        aspect = spacing[1] / spacing[0]

    # ======================= CORONAL =======================
    elif plane == 'Coronal':
        img1 = np.flipud(np.rot90(ct1[:, slice_index, :]))
        img2 = np.flipud(np.rot90(ct2[:, slice_index, :]))

        m1 = np.flipud(np.rot90(mand1[:, slice_index, :]))
        m2 = np.flipud(np.rot90(mand2[:, slice_index, :]))

        aspect = spacing[2] / spacing[0]

    # ======================= SAGITTAL =======================
    elif plane == 'Sagittal':
        img1 = np.flipud(np.rot90(ct1[slice_index, :, :]))
        img2 = np.flipud(np.rot90(ct2[slice_index, :, :]))

        m1 = np.flipud(np.rot90(mand1[slice_index, :, :]))
        m2 = np.flipud(np.rot90(mand2[slice_index, :, :]))

        aspect = spacing[2] / spacing[1]


    # ======================= BASE CT1 =======================
    plt.imshow(img1, cmap='gray', origin='lower', aspect=aspect)


    # ======================= CT2 bone overlay (intensity preserved!) =======================
    ct2_rgba = overlay_cmap(img2)
    ct2_rgba[..., 3] = alpha   # set transparency for entire CT2 layer
    plt.imshow(ct2_rgba, origin='lower', aspect=aspect)


    # ======================= Mandible 1 =======================
    overlay1 = np.zeros((*m1.shape, 4))
    overlay1[m1] = (*color_mand1[:3], alpha)
    plt.imshow(overlay1, origin='lower', aspect=aspect)


    # ======================= Mandible 2 =======================
    overlay2 = np.zeros((*m2.shape, 4))
    overlay2[m2] = (*color_mand2[:3], alpha)
    plt.imshow(overlay2, origin='lower', aspect=aspect)


    # ======================= Decoration =======================
    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")

    legend_handles = [
        plt.Line2D([0], [0], color=color_mand1[:3], lw=4, label="Mandible 1"),
        plt.Line2D([0], [0], color=color_mand2[:3], lw=4, label="Mandible 2"),
        plt.Line2D([0], [0], color="gray", lw=4, label="CT1 (base)"),
        plt.Line2D([0], [0], color="black", lw=4, label="CT2 (bone colormap)"),
    ]
    plt.legend(handles=legend_handles, loc='upper right', fontsize=8, frameon=False, labelcolor = "white")

    clear_output(wait=True)
    display(plt.gcf())
    plt.close()



# ---------------------- Widgets ----------------------
plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=ct1.shape[2] // 2,
    min=0,
    max=ct1.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


# update slice limits when plane changes
def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = ct1.shape[2] - 1
        slice_slider.value = ct1.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = ct1.shape[1] - 1
        slice_slider.value = ct1.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = ct1.shape[0] - 1
        slice_slider.value = ct1.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(
    show_slice,
    slice_index=slice_slider,
    plane=plane_dropdown,
    alpha=alpha_slider
)


interactive(children=(IntSlider(value=89, description='Slice:', layout=Layout(width='80%'), max=178), Dropdown…

<function __main__.show_slice(slice_index, plane, alpha=0.35)>